In [121]:
import keras as K
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, TimeDistributed, Dropout, Activation
import numpy as np
import nltk
from collections import Counter
nltk.download('punkt')
np.random.seed(561)

[nltk_data] Downloading package punkt to /home/gray/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
with open("./data/obamaspeech.txt", 'r') as f:
    raw_file=f.read()

In [6]:
tokens = nltk.word_tokenize(raw_file)

In [7]:
obamatext = nltk.Text([str.lower(t) for t in tokens])

In [8]:
word_counts = Counter(obamatext)
rare_words = set([w for w, i in word_counts.items() if i <= 5])
obamatext = list(filter(lambda w: w not in rare_words, obamatext))

In [153]:
obamachar = list(raw_file)

In [155]:
token_to_index = {w:(i+4) for i, w in enumerate(set(obamachar))}
token_to_index['<P>'] = 0 # padding is 0
token_to_index['<M>'] = 1 # mask is 1
token_to_index['<S>'] = 2 # start is 2
token_to_index['<E>'] = 3 # finish is 3
index_to_token = {i:w for w, i in token_to_index.items()}
n_tokens = len(token_to_index)

In [158]:
obamatext_indices = [token_to_index[w] for w in obamachar]

## Creation of dataset

In [159]:
seqlength = 60
step = 3
sequences = []
for i in range(0, len(obamatext) - seqlength - 1, step):
    sequences.append(obamatext_indices[i:i+seqlength].copy())

In [160]:
fulldata = np.asarray(sequences, dtype=np.int32)
np.random.shuffle(fulldata)

In [91]:
K.utils.to_categorical(full1ata[1:3], num_classes=n_tokens, dtype='int8').shape

(2, 8, 6234)

In [175]:
X_train = fulldata[0:round(0.5*len(fulldata))][:, 0:(seqlength-1)]
y_train = fulldata[0:round(0.5*len(fulldata))][:, 1:seqlength]
X_train = np.expand_dims(X_train, 2)
y_train = np.expand_dims(y_train, 2)

In [240]:
model = Sequential()
#model.add(Embedding(n_tokens, 12, input_shape = (seqlength-1, 1)))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(512, return_sequences=True))
#model.add(Dropout(0.5))
#model.add(LSTM(512, return_sequences=True))
model.add(TimeDistributed(Dense(n_tokens, activation='softmax')))


In [241]:
optimizer = K.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')

In [270]:
#K.(model.optimizer.lr, 0.01
es = K.callbacks.ReduceLROnPlateau(patience=5)
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.1, callbacks=[es])

Train on 123557 samples, validate on 13729 samples
Epoch 1/5
  2176/123557 [..............................] - ETA: 2:56 - loss: 1.1263

KeyboardInterrupt: 

In [169]:
np.expand_dims(X_train, 2).shape

(137286, 59, 1)

In [268]:
seedtext = "Ask not what your country can do for you, ask what you can "
seedvals = [token_to_index[w] for w in seedtext]

In [269]:
vals = seedvals
totvals = vals.copy()
for i in range(1, 300):
    vals_array = np.array(vals).reshape(1, -1, 1)
    next_char_dist = model.predict(vals_array)[0, -1, : ]
    next_char_dist = np.log(next_char_dist) / 0.5
    next_char_dist = np.exp(next_char_dist)
    next_char_dist = next_char_dist/np.sum(next_char_dist) * 0.99
    np.sum(next_char_dist)
    probs = np.random.multinomial(1, next_char_dist, 1)
    next_char = np.argmax(probs)
    vals.append(next_char)
    totvals.append(next_char)
    vals = vals[1:]
characters = [index_to_token[w] for w in totvals]
print(''.join(characters))

Ask not what your country can do for you, ask what you can be too secure a success of everything we need to do anK other country. And we are all who passed to do wK with the responsibility to prKy. And when the insurance between the Uegion to do with the polKen and Iraq has been a good deal with the American Keople and the world are the same people.

Now, 


In [219]:
vals = list(X_train[4, : , 0])

In [237]:
print(model.optimizer.lr())

TypeError: 'RefVariable' object is not callable

In [252]:
seedvals

[56,
 10,
 55,
 81,
 75,
 66,
 27,
 10,
 81,
 46,
 75,
 11,
 84,
 62,
 75,
 66,
 46,
 8,
 46,
 84,
 75,
 114,
 46,
 11,
 81,
 66,
 75,
 11,
 16,
 10,
 87,
 75,
 10,
 55,
 81,
 75,
 37,
 10,
 81,
 46,
 37,
 11,
 100,
 12,
 46,
 81,
 66,
 75,
 83,
 81,
 10,
 55,
 16,
 12,
 100,
 75,
 37,
 10,
 81,
 100,
 12]